In [1]:
import torch
from transformers import GenerationConfig
from load_model import load_model

f:\Miniconda3\envs\llm\lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
f:\Miniconda3\envs\llm\lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
# clone 模型
PRETRAINED_MODEL_NAME_OR_PATH = '../models/internlm2-chat-1_8b'
# os.system(f'git clone https://code.openxlab.org.cn/OpenLMLab/internlm2-chat-1.8b {PRETRAINED_MODEL_NAME_OR_PATH}')
# os.system(f'cd {PRETRAINED_MODEL_NAME_OR_PATH} && git lfs pull')
ADAPTER_PATH = None
# 量化
LOAD_IN_8BIT= False
LOAD_IN_4BIT = False

In [3]:
tokenizer, model = load_model(PRETRAINED_MODEL_NAME_OR_PATH, ADAPTER_PATH, LOAD_IN_8BIT, LOAD_IN_4BIT)

torch version:  2.2.2+cu121
transformers version:  4.36.2


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model.device: cuda:0, model.dtype: torch.float16


In [4]:
tokenizer

InternLM2TokenizerFast(name_or_path='../models/internlm2-chat-1_8b', vocab_size=92544, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '</s>', 'additional_special_tokens': ['<|im_start|>', '<|im_end|>', '<|action_start|>', '<|action_end|>', '<|interpreter|>', '<|plugin|>']}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	92538: AddedToken("<|plugin|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	92539: AddedToken("<|interpreter|>", rstrip=False, lstrip=False, single_word=False, no

In [16]:
print(tokenizer.bos_token_id)
print(tokenizer.eos_token_id)
print(tokenizer.convert_tokens_to_ids(["<|im_start|>", "<|im_end|>"]))

1
2
[92543, 92542]


In [5]:
model

InternLM2ForCausalLM(
  (model): InternLM2Model(
    (tok_embeddings): Embedding(92544, 2048, padding_idx=2)
    (layers): ModuleList(
      (0-23): 24 x InternLM2DecoderLayer(
        (attention): InternLM2Attention(
          (wqkv): Linear(in_features=2048, out_features=4096, bias=False)
          (wo): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): InternLM2RotaryEmbedding()
        )
        (feed_forward): InternLM2MLP(
          (w1): Linear(in_features=2048, out_features=8192, bias=False)
          (w3): Linear(in_features=2048, out_features=8192, bias=False)
          (w2): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (attention_norm): InternLM2RMSNorm()
        (ffn_norm): InternLM2RMSNorm()
      )
    )
    (norm): InternLM2RMSNorm()
  )
  (output): Linear(in_features=2048, out_features=92544, bias=False)
)

In [6]:
SYSTEM_PROMPT = """You are an AI assistant whose name is InternLM (书生·浦语).
    - InternLM (书生·浦语) is a conversational language model that is developed by Shanghai AI Laboratory (上海人工智能实验室). It is designed to be helpful, honest, and harmless.
    - InternLM (书生·浦语) can understand and communicate fluently in the language chosen by the user such as English and 中文.
    """

In [ ]:
help(tokenizer)

Help on InternLM2TokenizerFast in module transformers_modules.internlm2-chat-1_8b.tokenization_internlm2_fast object:

class InternLM2TokenizerFast(transformers.tokenization_utils_fast.PreTrainedTokenizerFast)
 |  InternLM2TokenizerFast(vocab_file, unk_token='<unk>', bos_token='<s>', eos_token='</s>', pad_token='</s>', sp_model_kwargs: Optional[Dict[str, Any]] = None, add_bos_token=True, add_eos_token=False, decode_with_prefix_space=False, clean_up_tokenization_spaces=False, **kwargs)
 |  
 |  # Modified from transformers.model.llama.tokenization_llama_fast.LlamaTokenizerFast -> InternLM2TokenizerFast
 |  
 |  Method resolution order:
 |      InternLM2TokenizerFast
 |      transformers.tokenization_utils_fast.PreTrainedTokenizerFast
 |      transformers.tokenization_utils_base.PreTrainedTokenizerBase
 |      transformers.tokenization_utils_base.SpecialTokensMixin
 |      transformers.utils.hub.PushToHubMixin
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self,

In [7]:
# https://huggingface.co/internlm/internlm2-chat-1_8b/blob/main/modeling_internlm2.py#L1136
def build_inputs(tokenizer, query: str, history: list[tuple[str, str]] = [], meta_instruction=""):
    if tokenizer.add_bos_token:
        prompt = ""
    else:
        prompt = tokenizer.bos_token
    if meta_instruction:
        prompt += f"""<|im_start|>system\n{meta_instruction}<|im_end|>\n"""
    for record in history:
        prompt += f"""<|im_start|>user\n{record[0]}<|im_end|>\n<|im_start|>assistant\n{record[1]}<|im_end|>\n"""
    prompt += f"""<|im_start|>user\n{query}<|im_end|>\n<|im_start|>assistant\n"""
    return prompt, tokenizer([prompt], return_tensors="pt")

In [8]:
prompt, inputs = build_inputs(tokenizer, "给我讲一个猫和老鼠的小故事", history=[], meta_instruction=SYSTEM_PROMPT)
print(prompt)

<|im_start|>system
You are an AI assistant whose name is InternLM (书生·浦语).
    - InternLM (书生·浦语) is a conversational language model that is developed by Shanghai AI Laboratory (上海人工智能实验室). It is designed to be helpful, honest, and harmless.
    - InternLM (书生·浦语) can understand and communicate fluently in the language chosen by the user such as English and 中文.
    <|im_end|>
<|im_start|>user
给我讲一个猫和老鼠的小故事<|im_end|>
<|im_start|>assistant



In [9]:
inputs = inputs.to(model.device)
print("input_ids: ", inputs["input_ids"]) # 开始为1,添加了 bos_token_id
print("attention_mask: ", inputs["attention_mask"])

input_ids:  tensor([[    1, 92543,  9081,   364,  2770,   657,   589, 15358, 17993,  6843,
           963,   505,  4576, 11146,   451, 60628, 60384, 60721, 62442, 60752,
          4452,   388,   285,  4576, 11146,   451, 60628, 60384, 60721, 62442,
         60752,   313,   505,   395,  7659,  1813,  4287,  1762,   560,   505,
          8020,   684, 36956, 15358, 31288,   451, 68589, 76659, 71581,   699,
          1226,   505,  6342,   442,   517, 11100,   328, 10894,   328,   454,
         51978,   756,   388,   285,  4576, 11146,   451, 60628, 60384, 60721,
         62442, 60752,   313,   777,  3696,   454, 19187, 19829,  4563,   435,
           410,  4287, 12032,   684,   410,  1341,  1893,   569,  6519,   454,
           262, 69093,   756,   388, 92542,   364, 92543,  1008,   364, 68706,
         61077, 68252, 61519, 60381, 74262, 68447, 68654, 92542,   364, 92543,
           525, 11353,   364]], device='cuda:0')
attention_mask:  tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [34]:
generation_config = GenerationConfig(
    max_new_tokens = 1024,
    do_sample = True,
    num_beams = 1,
    temperature = 0.8,
    top_k = 40,
    top_p = 0.8,
    eos_token_id = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids(["<|im_end|>"])[0]]
)
generation_config

GenerationConfig {
  "do_sample": true,
  "eos_token_id": [
    2,
    92542
  ],
  "max_new_tokens": 1024,
  "temperature": 0.8,
  "top_k": 40,
  "top_p": 0.8
}

In [44]:
model.eval()
with torch.inference_mode():
    outputs = model.generate(
        input_ids = inputs["input_ids"],
        attention_mask = inputs["attention_mask"],
        generation_config = generation_config,
    )
# 输出是二维的
outputs

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


tensor([[    1, 92543,  9081,   364,  2770,   657,   589, 15358, 17993,  6843,
           963,   505,  4576, 11146,   451, 60628, 60384, 60721, 62442, 60752,
          4452,   388,   285,  4576, 11146,   451, 60628, 60384, 60721, 62442,
         60752,   313,   505,   395,  7659,  1813,  4287,  1762,   560,   505,
          8020,   684, 36956, 15358, 31288,   451, 68589, 76659, 71581,   699,
          1226,   505,  6342,   442,   517, 11100,   328, 10894,   328,   454,
         51978,   756,   388,   285,  4576, 11146,   451, 60628, 60384, 60721,
         62442, 60752,   313,   777,  3696,   454, 19187, 19829,  4563,   435,
           410,  4287, 12032,   684,   410,  1341,  1893,   569,  6519,   454,
           262, 69093,   756,   388, 92542,   364, 92543,  1008,   364, 68706,
         61077, 68252, 61519, 60381, 74262, 68447, 68654, 92542,   364, 92543,
           525, 11353,   364, 68529, 68251, 60355, 68262, 68654, 73126,   904,
         69253, 69368, 70511,   338,  1467,  1320,  

In [45]:
# 取出第一条数据
ids = outputs[0].cpu()[len(inputs["input_ids"][0]) :]
ids

tensor([68529, 68251, 60355, 68262, 68654, 73126,   904, 69253, 69368, 70511,
          338,  1467,  1320,   281, 60354, 70249, 76044, 60503, 61519, 60381,
        74262, 60501, 60355, 68654, 92155, 61519, 60381, 74262, 69713, 73459,
        73859, 60353, 68375, 68310, 68343, 68705, 69702, 76543, 71775, 80092,
        60355,   402, 61519, 78131, 60419, 61310, 61279, 60420, 60353, 74262,
        78131, 60419, 64917, 64917, 60420, 60355, 68310, 74360, 72951, 61653,
        64068, 60380, 60353, 61310, 61279, 68556, 61882, 60633, 64917, 64917,
        60354, 73503, 60353, 60458, 64917, 64917, 60395, 68965, 74375, 72477,
        61310, 61279, 60355, 68310, 69713, 70696, 68705, 69085, 60353, 69125,
        70619, 60353, 61310, 61279, 68792, 70714, 69786, 60355,   402, 61310,
        61279, 60379, 60404, 60462, 62900, 72607, 61310, 61279, 60361, 78559,
        60797, 68849, 70329, 60419, 61310, 61279, 60420, 60353, 60367, 60602,
        75824, 60355, 64917, 64917, 68335, 70974, 60353, 68792, 

In [46]:
# decode 处理一维数据
response = tokenizer.decode(ids, skip_special_tokens=True)
print(response)

当然可以。这个故事源于19世纪英国作家J.M.W.的著名童话《猫和老鼠》。故事描述了猫和老鼠之间的激烈斗争，以及他们如何不断尝试逃避对方的追逐。

猫名叫“胡须”，老鼠名叫“吱吱”。他们住在同一个屋檐下，胡须经常偷吃吱吱的粮食，而吱吱也总是试图抓住胡须。他们之间的矛盾不断升级，直到有一天，胡须决定采取行动。

胡须用他那锐利的胡须在墙上画了一个很大的“胡须”，以示警告。吱吱非常生气，决定亲自解决这个麻烦。他用他那尖利的牙齿咬破了胡须的胡须，然后逃到了他的洞里。

胡须感到非常失望和愤怒，他决定向吱吱复仇。他开始跟踪吱吱，并且经常在他的洞口周围制造噪音。吱吱感到非常害怕，他开始躲在墙角处，试图躲避胡须的追逐。

胡须继续跟踪吱吱，并不断在他身上留下自己的印记。他甚至开始用他的胡须抓挠吱吱的尾巴，试图引起他的注意。吱吱感到非常痛苦和绝望，他知道自己已经无法逃脱胡须的追捕。

最终，胡须找到了一个洞穴，他躲在那里，静静地等待着吱吱的到来。吱吱终于被胡须的胡须所吸引，他走进了洞穴，与胡须展开了一场激烈的追逐战。

在一场激烈的追逐中，胡须用他的胡须攻击了吱吱，但吱吱却巧妙地躲开了这些攻击。最终，胡须败给了吱吱，他不得不离开自己的洞穴，去寻找其他藏身之处。

这个故事告诉我们，即使是最强大和最狡猾的对手，只要我们保持冷静和机智，我们也能战胜它们。


In [47]:
# batch_decode 处理二维数据
print(tokenizer.batch_decode([ids], skip_special_tokens=True)[0])

当然可以。这个故事源于19世纪英国作家J.M.W.的著名童话《猫和老鼠》。故事描述了猫和老鼠之间的激烈斗争，以及他们如何不断尝试逃避对方的追逐。

猫名叫“胡须”，老鼠名叫“吱吱”。他们住在同一个屋檐下，胡须经常偷吃吱吱的粮食，而吱吱也总是试图抓住胡须。他们之间的矛盾不断升级，直到有一天，胡须决定采取行动。

胡须用他那锐利的胡须在墙上画了一个很大的“胡须”，以示警告。吱吱非常生气，决定亲自解决这个麻烦。他用他那尖利的牙齿咬破了胡须的胡须，然后逃到了他的洞里。

胡须感到非常失望和愤怒，他决定向吱吱复仇。他开始跟踪吱吱，并且经常在他的洞口周围制造噪音。吱吱感到非常害怕，他开始躲在墙角处，试图躲避胡须的追逐。

胡须继续跟踪吱吱，并不断在他身上留下自己的印记。他甚至开始用他的胡须抓挠吱吱的尾巴，试图引起他的注意。吱吱感到非常痛苦和绝望，他知道自己已经无法逃脱胡须的追捕。

最终，胡须找到了一个洞穴，他躲在那里，静静地等待着吱吱的到来。吱吱终于被胡须的胡须所吸引，他走进了洞穴，与胡须展开了一场激烈的追逐战。

在一场激烈的追逐中，胡须用他的胡须攻击了吱吱，但吱吱却巧妙地躲开了这些攻击。最终，胡须败给了吱吱，他不得不离开自己的洞穴，去寻找其他藏身之处。

这个故事告诉我们，即使是最强大和最狡猾的对手，只要我们保持冷静和机智，我们也能战胜它们。
